# 2024 COMP90042 Project
*Make sure you change the file name with your group id.*

# Readme
*If there is something to be noted for the marker, please mention here.*

*If you are planning to implement a program with Object Oriented Programming style, please put those the bottom of this ipynb file*

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [332]:
## package imports

#!pip install pandas scikit-learn torch torchtext
!pip3 install torchtext==0.4.0

## deep-learning libraries
import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F
import keras
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.data import Field, LabelField, Example, Dataset
from tensorflow import convert_to_tensor

## NLP preprocessing libraries
import nltk
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer

##others
import pandas as pd
import numpy as np
import matplotlib as plt
import string
import json



In [333]:
## reading in json files

"""
Description of json files
* [train-claims,dev-claims].json: JSON files for the labelled training and development set; 
* evidence.json: JSON file containing a large number of evidence passages (i.e. the “knowledge source”); 
* dev-claims-baseline.json: JSON file containing predictions of a baseline system on the development set;
"""

## relative file paths

## baseline system - will not be used for any training/evaluation
devClaimsBaselineFile='./data/dev-claims-baseline.json'
## use this for model training
trainClaimsFile='./data/train-claims.json'
## use this set for hyperparameter tuning and evaluation metric 
devClaimsFile='./data/dev-claims.json'
## evidence files need to be downloaded through https://drive.google.com/file/d/1JlUzRufknsHzKzvrEjgw8D3n_IRpjzo6/view?usp=sharing as it is to big to be uploaded to github
evidenceFile='./data/evidence.json'
## test unlabelled dataset
testFile='./data/test-claims-unlabelled.json'

In [334]:
# Load the JSON data
with open(trainClaimsFile, 'r') as file:
    trainClaims=json.load(file)
with open(devClaimsFile, 'r') as file:
    devClaims=json.load(file)
with open(evidenceFile, 'r') as file:
    evidenceData=json.load(file)

## Preprocessing data -- lowercase, tokenize, and stopword removal
stopwords=set(nltk.corpus.stopwords.words('english'))
tokenizer=get_tokenizer('basic_english')
punctuations=string.punctuation

def preprocess(text):
    token=tokenizer(text.lower())
    cleanedTokens=[t for t in token if (t not in stopwords) and (t not in punctuations)]
    return ' '.join(cleanedTokens)

for ids, texts in evidenceData.items():
    evidenceData[ids]=preprocess(texts)

# Function to create DataFrame and merge evidence IDs with Text
def createDF(claims, evidence):
    combinedData=[]
    for claimID, claimText in claims.items():
        # Combine the ID with its corresponding evidences
        evidenceID=claimText['evidences']
        evidenceText=(evidence[i] for i in evidenceID if i in evidence)
        combinedData.append({
            'claim_id': claimID,
            'claim_text': preprocess(claimText['claim_text']),
            'evidence_id': evidenceID,
            'evidence_text': " ".join(evidenceText),
            'claim_label': claimText['claim_label']
        })
    # Create DataFrame
    return pd.DataFrame(combinedData)

# Create CSV Files
trainFullMerged=createDF(trainClaims,evidenceData)
devFullMerged=createDF(devClaims,evidenceData)
trainFullMerged.to_csv("data/trainFullMerged.csv", index=False)
devFullMerged.to_csv("data/devFullMerged.csv", index=False)

# Convert evidence into csv as well
evidenceFinal=pd.DataFrame(list(evidenceData.items()),columns=['evidence_id','evidence_text'])
evidenceFinal.to_csv('data/evidencePreprocessed.csv',index=False)


In [335]:
# Convert unlabelled Data into CSV as well
with open(testFile, 'r') as file:
    testData=json.load(file)
for ids, texts in testData.items():
    claim_text = texts['claim_text']
    testData[ids]=preprocess(claim_text)
testFinal=pd.DataFrame(list(testData.items()), columns=['claim_id', 'claim_text'])
testFinal.to_csv('data/testPreprocessed.csv',index=False)

In [338]:
# Reading in the created CSV files

## used this for setting how much to see when printing dataframes, 50 is default
pd.set_option('display.max_colwidth', None)

## use this for model training
trainClaimsFile='./data/trainFullMerged.csv'
## use this set for hyperparameter tuning and evaluation metric 
devClaimsFile='./data/devFullMerged.csv'
## evidence files need to be downloaded through googledrive (https://drive.google.com/file/d/1OyihwdAWfqHIOueCB4bLBkYg4hTN_OKm/view?usp=sharing)
evidenceFile='./data/evidencePreprocessed.csv'
## test unlabelled dataset
testFile='./data/testPreprocessed.csv'

trainDataframe=pd.read_csv(trainClaimsFile)
devDataframe=pd.read_csv(devClaimsFile)
evidenceDataframe=pd.read_csv(evidenceFile)

## One consideration is whether we want the evidence_id at all

trainDataframe['evidence_id']=trainDataframe['evidence_id'].astype(str).str.strip('[]').str.strip("'")
trainDataframe['combined_evidence']=trainDataframe['evidence_id']+" "+trainDataframe['evidence_text']
devDataframe['evidence_id']=devDataframe['evidence_id'].astype(str).str.strip('[]').str.strip("'")
devDataframe['combined_evidence']=devDataframe['evidence_id'] +" "+ devDataframe['evidence_text']
evidenceDataframe['combined_evidence']=evidenceDataframe['evidence_id']+" "+evidenceDataframe['evidence_text']
evidenceDataframe['combined_evidence']=evidenceDataframe['combined_evidence'].astype(str).str.strip("'")

print('shape of training set:',trainDataframe.shape,trainDataframe.columns)
print('shape of development set:',devDataframe.shape,devDataframe.columns)
print('shape of evidence:',evidenceDataframe.shape,evidenceDataframe.columns)


TEXT=torchtext.data.Field(tokenize=preprocess,
                          init_token='<text1>',
                          eos_token='<text2>',
                          lower=True)
LABEL = torchtext.data.LabelField(tokenize=preprocess,
                          lower=True)

## This converts dataframes into tensors, it does preprocessing twice as the data is already preprocessed but will leave it as is for now
def createDatasetEncoderInput(dataframe,textTransform,labelTransform):
    fields=[(('reviewTextInput'),textTransform),(('evidenceTextOutput'),labelTransform)]
    examples=[]
    for _, row in dataframe.iterrows():
        reviewTextInput=row['claim_text']
        evidenceTextOutput=row['combined_evidence']
        examples.append(Example.fromlist([reviewTextInput,evidenceTextOutput], fields))
    return Dataset(examples,fields)

trainTensor=createDatasetEncoderInput(trainDataframe,TEXT,LABEL)
devTensor=createDatasetEncoderInput(devDataframe,TEXT,LABEL)
    
## now, start to create the decoder inputs in the form of DATASETS for the combined_evidence
def createDatasetDecoderInput(dataframe,labelTransform):
    fields=[(('evidenceTextOutput'),labelTransform)]
    examples=[]
    for _, row in dataframe.iterrows():
        evidenceTextOutput=row['combined_evidence']
        examples.append(Example.fromlist([evidenceTextOutput], fields))
    return Dataset(examples,fields)

evidenceTensor=createDatasetDecoderInput(evidenceDataframe,LABEL)

ValueError: If using all scalar values, you must pass an index

In [337]:
assert (len(trainDataframe)==len(trainTensor)) and (len(devDataframe)==len(devTensor) and (len(evidenceDataframe)==len(evidenceTensor)))

# checking how the tensors look
for i in range(min(len(trainTensor), 3)):
    print('train',vars(trainTensor.examples[i]))
    print('dev',vars(devTensor.examples[i]))
    print('evidence',vars(evidenceTensor.examples[i]))

train {'reviewTextInput': 'scientific evidence co2 pollutant higher co2 concentrations actually help ecosystems support plant animal life', 'evidenceTextOutput': "evidence-442946', 'evidence-1194317', 'evidence-12171 high concentrations 100 times atmospheric concentration greater carbon dioxide toxic animal life raising concentration 10 000 ppm 1% higher several hours eliminate pests whiteflies spider mites greenhouse plants grow much 50 percent faster concentrations 1 000 ppm co 2 compared ambient conditions though assumes change climate limitation nutrients higher carbon dioxide concentrations favourably affect plant growth demand water"}
dev {'reviewTextInput': '[south australia] expensive electricity world', 'evidenceTextOutput': "evidence-67732', 'evidence-572512 [citation needed] south australia highest retail price electricity country south australia highest power prices world"}
evidence {'evidenceTextOutput': 'evidence-0 john bennet lawes english entrepreneur agricultural scien

# 2. Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

# 3.Testing and Evaluation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

## Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed*